# Plot the data
Used to map the COVID-19 cases of Germany on a map or plotting the number of cases/seven days incidence of one county.
<br/><br/><br/>
Provides the dictionary "counties_geography" with different kinds of geographical information about every German county. For more information check out the file "get_geographical_data_of_german_counties.ipynb".
<br/><br/>
Provides the accumulated COVID-19 case number and the seven days incidences of every german county and every day of the pandemic in the dictionary "covid19" (reachable by the county's AdmUnitID).
For more information check out the file "get_data.ipynb" (which calls the file "get_geographical_data_of_german_counties.ipynb" if needed).

## Modules
Needed to use non-Python functionalities already programmed by someone else.

In [ ]:
# Used to cast the polygons into np.arrays and afterwards transpose them easily with the .T function
import numpy as np
import matplotlib.pyplot as plt    # to plot the counties
# used to determine which part of the plot shall be saved
from matplotlib.transforms import Bbox

## Get the data
More information about the data can be find inside the first cell.

In [ ]:
%run get_data.ipynb
print("All data is ready!")

## Create color schemes

In [ ]:
red_yellow_green = list()
# bigger steps in the darkgreen spectrum to make a bigger optical difference between
# smaller numbers (e.g. 7 and 8) in comparison to bigger numbers (e.g. 57 and 58)
# (most of the time most counties are green and hard to distinguish and only a few are red)
for color_step in np.arange(0.3,1,0.05):
    red_yellow_green.append((0, color_step, 0))
for color_step in np.arange(0,1,0.02):
    red_yellow_green.append((color_step, 1, 0))
for color_step in np.arange(0,1,0.01):
    red_yellow_green.append((1, 1 - color_step, 0))
for color_step in np.arange(0,1,0.005):
    red_yellow_green.append((1 - color_step, 0, 0))
font = dict(family= 'serif', color= 'black', weight= 'bold', size= 90)

In [ ]:
# check if the needed directory is availlable - otherwise create it
if not(os.path.isdir("germany_incidence")): os.makedirs("germany_incidence")

for date_index in range(345, len(non_county_specific_data['UTC'])):
    fig = plt.figure(figsize = (50, 50), facecolor='darkgrey')
    ax = plt.axes(frameon=False)
    for AdmUnitID, county in covid19.items():
        color_index_incidence = int(county['incidences'][date_index])
        if color_index_incidence >= len(red_yellow_green):
            color = red_yellow_green[-1]
        else:
            color = red_yellow_green[color_index_incidence]
        for polygon in counties_geography[AdmUnitID]['geometry']:
            x,y = np.array(polygon).T
            plt.fill(x, y, color=color, facecolor="darkgrey")
        # draw the lines around the counties
        for polygon in counties_geography[AdmUnitID]['raw_geometry']:
            x,y = np.array(polygon).T
            plt.plot(x, y, color="black")
    plt.title("Incidences in Germany on day " + 
              str(non_county_specific_data['UTC'][date_index])[:10],
              fontdict=font)
    plt.savefig("germany_incidence/" + str(date_index) + '.png',
                facecolor="darkgrey", bbox_inches=Bbox([[7,7],[44,46]]))
    plt.close('all')

In [ ]:
g = list()
g[1]

In [ ]:
fig1 = plt.figure(facecolor='darkgrey', figsize = (20, 20))
ax1 = plt.axes(frameon=False)
ax1.axes.get_yaxis().set_visible(False)
ax1.axes.get_xaxis().set_visible(False)
for AdmUnitID, county in covid19.items():
    # get last incidence and divide it by the highest incidence to get a number between 0 and 1
    # to get the full range of colors the both incidence values get subtracted by the lowest incidence
    color_step = county['incidences'][-1]/250
    # everything bellow 0.5 gets drawn green
    if color_step > 1:
        color = (1, 1, 0)
    elif color_step < 0.2:
        color = (color_step * 5, 1, 0)
    else:
        color_step = (color_step - 0.2) * 1.25
        color = (1, 1 - color_step, 0)
        plt.title("Incidences in Germany on day " + str(non_county_specific_data['UTC'][-1])[:10], fontdict=font)
    for polygon in counties_geography[AdmUnitID]['geometry']:
        x,y = np.array(polygon).T
        plt.fill(x, y, color=color)
plt.show()

In [ ]:
for colorID in range(0, len(red_yellow_green), 20):
    for polygon in counties_geography["8311"]['geometry']:
        x,y = np.array(polygon).T
        plt.fill(x, y, color=red_yellow_green[colorID])
        plt.plot(x, y, color=(0,0,0))
    plt.show()

In [ ]:
red_to_yellow_to_green = list()
for color_step in np.arange(0,1,0.01):
    red_to_yellow_to_green.append((color_step, 1, 0))
for color_step in np.arange(0,1,0.01):
    red_to_yellow_to_green.append((1, 1 - color_step, 0))

In [ ]:
for colorID in range(0, len(red_to_yellow_to_green), 15):
    for polygon in counties_geography["8311"]['geometry']:
        x,y = np.array(polygon).T
        plt.fill(x, y, color=red_to_yellow_to_green[colorID])
        plt.plot(x, y, color=(0,0,0))
    plt.show()

In [ ]:
blue_to_white_to_red = list()
for color_step in np.arange(0,1,0.01):
    blue_to_white_to_red.append((color_step, color_step, 1))
for color_step in np.arange(0,1,0.01):
    blue_to_white_to_red.append((1, 1 - color_step, 1 - color_step))

In [ ]:
for colorID in range(0, len(blue_to_white_to_red), 15):
    for polygon in counties_geography["8311"]['geometry']:
        x,y = np.array(polygon).T
        plt.fill(x, y, color=blue_to_white_to_red[colorID])
        plt.plot(x, y, color=(0,0,0))
    plt.show()

## Plot examples

In [ ]:
# copy to check that all counties are related to a federal state
covid19_copy = covid19.copy()
for stateID, state in non_county_specific_data['states'].items():
    plt.figure(figsize=(20, 5))    # enlarge plot
    plt.title("Incidences in {} {}".format(stateID, state))
    for AdmUnitID, county in covid19.items():
        if AdmUnitID[:-3] == stateID:
            plt.plot(non_county_specific_data['UTC'], county['incidences'])
            del covid19_copy[AdmUnitID]
    plt.show()

In [ ]:
if len(covid19_copy) != 0:
    print('There seems to be at least one county being unrelated to any federal state:')
    for AdmUnitID in covid19_copy.keys():
        print(AdmUnitID + " " + counties_geography[AdmUnitId]['name'])

## Plot data of one state

In [ ]:
example_state_ID = "13"
print("You have chosen {} {}".format(example_state_ID, non_county_specific_data['states'][example_state_ID]))

In [ ]:
counties_in_example_state = list()
for AdmUnitID, county in covid19.items():
    if AdmUnitID[:-3] == "13":
        counties_in_example_state.append(AdmUnitID)
        plt.figure(figsize=(20, 5))    # enlarge plot
        plt.plot(non_county_specific_data['UTC'], county['incidences'])
        plt.title("Incidences in " + AdmUnitID + " " + counties_geography[AdmUnitID]['name'])
        plt.show()

In [ ]:
plt.figure(figsize = (20, 20))    # enlarge plot
for AdmUnitID in counties_in_example_state:
    # get last incidence and divide it by the highest incidence to get a number between 0 and 1
    # to get the full range of colors the both incidence values get subtracted by the lowest incidence
    color_step = covid19[AdmUnitID]['incidences'][-1]/250
    # everything bellow 0.5 gets drawn green
    if color_step > 1:
        color = (1, 1, 0)
    elif color_step < 0.2:
        color = (color_step * 5, 1, 0)
    else:
        color_step = (color_step - 0.2) * 1.25
        color = (1, 1 - color_step, 0)
    print(color)
    for polygon in counties_geography[AdmUnitID]['geometry']:
        x,y = np.array(polygon).T
        plt.fill(x, y, color=color)
plt.show()

## Normating the color schemes
Aiming for an even distribution between all color fields

In [ ]:
plt.figure(figsize = (20, 20))    # enlarge plot
distribution_of_incidences = [0 for e in range(400)]
for county in covid19.values():
    for incidence in county['incidences']:
        if int(incidence) >= 250:
            print(int(incidence))
        else:
            distribution_of_incidences[int(incidence)] += 1
plt.plot([e + 25 for e in range(375)], distribution_of_incidences[25:])
plt.show